<a href="https://colab.research.google.com/github/UshaSDiggi/UshaSDiggi/blob/main/AutoEncodeModel_Denoising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras import layers, losses
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
(x_train, _),(x_test, _) = fashion_mnist.load_data()
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
x_train = x_train[...,tf.newaxis]
x_test = x_test[...,tf.newaxis]
noise_factor = 0.2
x_train_noisy = x_train + noise_factor * tf.random.normal(shape=x_train.shape) 
x_test_noisy = x_test + noise_factor * tf.random.normal(shape=x_test.shape) 

x_train_noisy = tf.clip_by_value(x_train_noisy, clip_value_min=0., clip_value_max=1.)
x_test_noisy = tf.clip_by_value(x_test_noisy, clip_value_min=0., clip_value_max=1.)

In [ ]:

class Denoise(Model):
  def __init__(self):
    super(Denoise,self).__init__()
    self.encoder = tf.keras.Sequential([layers.Input(shape=(28,28,1)),
                                        layers.Conv2D(16,(3,3),activation='relu',padding='same',strides=2),
                                        layers.Conv2D(8,(3,3),activation='relu',padding='same',strides=2)
                                        ])
    self.decoder = tf.keras.Sequential([
                                        layers.Conv2DTranspose(8,kernel_size=3,strides=2,activation='relu',padding='same'),
                                        layers.Conv2DTranspose(16,kernel_size=3,strides=2,activation='relu',padding='same'),
                                        layers.Conv2D(1,kernel_size=(3,3),activation='sigmoid',padding='same')])
  def call(self,x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded


In [ ]:
autoencoder = Denoise()
autoencoder.compile(optimizer='adam',loss=losses.MeanSquaredError())

In [ ]:
autoencoder.fit(x_train_noisy,x_train,
                epochs=10,
                shuffle=True,
                validation_data=(x_test_noisy,x_test))

Epoch 1/10
1875/1875 [==============================] - 71s 37ms/step - loss: 0.0175 - val_loss: 0.0096
Epoch 2/10
1875/1875 [==============================] - 68s 36ms/step - loss: 0.0090 - val_loss: 0.0087
Epoch 3/10
1875/1875 [==============================] - 68s 36ms/step - loss: 0.0081 - val_loss: 0.0078
Epoch 4/10
1875/1875 [==============================] - 68s 36ms/step - loss: 0.0077 - val_loss: 0.0076
Epoch 5/10
1875/1875 [==============================] - 70s 37ms/step - loss: 0.0075 - val_loss: 0.0075
Epoch 6/10
1875/1875 [==============================] - 72s 39ms/step - loss: 0.0073 - val_loss: 0.0071
Epoch 7/10
1875/1875 [==============================] - 71s 38ms/step - loss: 0.0070 - val_loss: 0.0070
Epoch 8/10
1875/1875 [==============================] - 68s 36ms/step - loss: 0.0069 - val_loss: 0.0070
Epoch 9/10
1875/1875 [==============================] - 73s 39ms/step - loss: 0.0069 - val_loss: 0.0069
Epoch 10/10
1875/1875 [==============================] - 70s 38m

In [ ]:
encoded_imgs = autoencoder.encoder(x_test).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()